Capítulo 6

6.4 Patrones de unión.

6.4.6 Unión replicada 

Prototipo en el que cargamos la primera tabla en memoria, y posteriomente metemos por línea de comandos todos las tablas que queramos unir a esa primera tabla. Las uniones que se producen pueden ser internas y/o externas por la izquierda. 

In [11]:
%%writefile unionReplicada.py
#!/usr/bin/python
from mrjob.job import MRJob
import re

def cargarFicheroEnMemoria(self,fichero):
    diccionario={}
    with open(fichero) as f:
        self.tablaEnMemoria = set(line.strip() for line in f)
    for linea in self.tablaEnMemoria:
        #Para que no tenga en cuenta las cabeceras de las tablas
        encontrado=re.search('[a-zA-Z]',linea[0])
        if encontrado==None:
            datos = linea.split(";")
            diccionario[datos[0]]=datos
    return diccionario

class unionReplicada(MRJob):
   
    #FILES = ['hdfs:///archivos_datos/tiendas.csv'] #Para ejecución en el CLUSTER
    FILES = ['archivos_datos/tiendas-articulos/tiendas.csv'] # Para ejecución en local
    fichero='tiendas.csv'    
    def mapper_init(self):
        #Nos devuelve la estructura diccionario rellena con los datos del fichero
        self.dicTablaA=cargarFicheroEnMemoria(self,self.fichero) 
       
    def mapper(self,_,line):
        self.linea=line.split(';')
        encontrado=re.search('[a-zA-Z]',self.linea[0])
        if encontrado==None:
            self.clave=self.linea[1] #Compara Clave de la tabla/s del stream 
            if self.clave in self.dicTablaA: 
                yield self.clave,(self.linea,self.dicTablaA.get(self.clave) )
            else:
                yield self.clave,(self.linea,"null")
                
if __name__ == '__main__':
    unionReplicada.run()

Overwriting unionReplicada.py


In [2]:
!python unionReplicada.py archivos_datos/tiendas-articulos/articulos_stock.csv archivos_datos/tiendas-articulos/articulos_stock2.csv archivos_datos/tiendas-articulos/articulos_stock3.csv

FILES: archivos_datos/tiendas-articulos/tiendas.csv will appear in same directory as job script, not a subdirectory
No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/unionReplicada.root.20201214.230312.067818
Running step 1 of 1...
job output is in /tmp/unionReplicada.root.20201214.230312.067818/output
Streaming final output from /tmp/unionReplicada.root.20201214.230312.067818/output...
"3"	[["1028", "3", "61"], ["3", "Granada"]]
"4"	[["1029", "4", "2"], "null"]
"5"	[["1030", "5", "67"], "null"]
"3"	[["1011", "3", "16"], ["3", "Granada"]]
"5"	[["1012", "5", "2"], "null"]
"2"	[["1013", "2", "17"], ["2", "Albacete"]]
"1"	[["1014", "1", "9"], ["1", "Santander"]]
"2"	[["1015", "2", "7"], ["2", "Albacete"]]
"3"	[["1016", "3", "2"], ["3", "Granada"]]
"5"	[["1017", "5", "1"], "null"]
"2"	[["1018", "2", "9"], ["2", "Albacete"]]
"1"	[["1019", "1", "13"], ["1", "Santander"]]
"1"	[["1020", "1", "23"], ["1", "Santander"]]
"5"	[

EJECUCIÓN EN EL CLUSTER

In [12]:
!python unionReplicada.py hdfs:///archivos_datos/articulos_stock.csv hdfs:///archivos_datos/articulos_stock2.csv hdfs:///archivos_datos/articulos_stock3.csv -r hadoop --python-bin /opt/anaconda/bin/python3.7

FILES: hdfs:///archivos_datos/tiendas.csv will appear in same directory as job script, not a subdirectory
No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /usr/lib/hadoop/bin...
Found hadoop binary: /usr/lib/hadoop/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /usr/lib/hadoop...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/unionReplicada.root.20201214.231725.899226
uploading working dir files to hdfs:///user/root/tmp/mrjob/unionReplicada.root.20201214.231725.899226/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/unionReplicada.root.20201214.231725.899226/files/
Running step 1 of 1...
  packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob6752507925837337719.jar tmpDir=null
  Connecting to ResourceManag